In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno 

from importlib import reload

#import process_gini as pg
#import plots
#import model as m

# Is investing in social media advertisement worth it?

1. [About the data](#About-the-data)
2. [Data Analysis](#Data-Analysis)
   1. [Subsection A](#Subsection-A)
   2. [Subsection B](#Subsection-B)
3. [Conclusion](#Conclusion)

Before getting into the results. Let's look at the given dataset

## About the data